Google Stocks

In [98]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet, set_log_level
import math

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact_manual

plotting_backend = "plotly-static"

In [3]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/multivariate/ER_Europe_subset_10nodes.csv"
)
df["ID"] = df["ID"].astype(str)
df["ds"] = pd.to_datetime(df["ds"])
# # use one year for faster training
df = df[df["ID"] =="1"]
df=df.drop(columns=["solar","solar_fcs","ID"])
df.head()

,ds,y
0,2012-01-01 00:00:00,75.6549
1,2012-01-01 01:00:00,70.9958
2,2012-01-01 02:00:00,66.6382
3,2012-01-01 03:00:00,62.9920
4,2012-01-01 04:00:00,61.0699


In [4]:
df.shape

(26304, 2)

Distrubution plots

In [6]:
fig = px.line(df, x=df["ds"], y=df["y"])
fig.show()

## ECG Data

Dataset for ECG taken from [Physio Bank ATM](https://archive.physionet.org/cgi-bin/atm/ATM) from the dataset [The MIT-BIH Long Term Database](https://archive.physionet.org/physiobank/database/ltdb/)

In [31]:
import pandas as pd

In [99]:
df=pd.read_csv("ECG_preds.csv")
df = df.iloc[1:].reset_index(drop=True)
df.head()

,'Elapsed time','ECG1','ECG2'
0,'0:00.000',0.160,-0.290
1,'0:00.008',0.180,-0.300
2,'0:00.016',0.170,-0.320
3,'0:00.023',0.180,-0.290
4,'0:00.031',0.180,-0.310


In [100]:
df_cleaned = df.rename(columns=lambda x: x.strip("'"))  
df_cleaned.head()

,Elapsed time,ECG1,ECG2
0,'0:00.000',0.160,-0.290
1,'0:00.008',0.180,-0.300
2,'0:00.016',0.170,-0.320
3,'0:00.023',0.180,-0.290
4,'0:00.031',0.180,-0.310


In [101]:
df_cleaned.ECG1 = df_cleaned.ECG1.astype('float64')
df_cleaned.ECG2 = df_cleaned.ECG2.astype('float64')

In [102]:
from datetime import datetime, timedelta
df_cleaned=df_cleaned.applymap(lambda x: x.strip("'") if isinstance(x, str) else x)  # Remove quotes from data
df_cleaned['Elapsed time'] = "00:0" + df_cleaned["Elapsed time"].astype(str) # Making it into hh:mm:ss format 
df_cleaned['Elapsed time'] = df_cleaned['Elapsed time'].apply(lambda x:timedelta(minutes=int(x.split(":")[1]), seconds=float(x.split(":")[2])))
df_cleaned.head()

,Elapsed time,ECG1,ECG2
0,0 days 00:00:00,0.16,-0.29
1,0 days 00:00:00.008000,0.18,-0.30
2,0 days 00:00:00.016000,0.17,-0.32
3,0 days 00:00:00.023000,0.18,-0.29
4,0 days 00:00:00.031000,0.18,-0.31


In [103]:
df_cleaned.head()

,Elapsed time,ECG1,ECG2
0,0 days 00:00:00,0.16,-0.29
1,0 days 00:00:00.008000,0.18,-0.30
2,0 days 00:00:00.016000,0.17,-0.32
3,0 days 00:00:00.023000,0.18,-0.29
4,0 days 00:00:00.031000,0.18,-0.31


In [104]:
start_time = datetime(2024, 1, 1, 0, 0, 0)
df_cleaned["Timestamp"] = start_time + df_cleaned["Elapsed time"]
df_cleaned.drop(columns=['Elapsed time'], inplace = True)
df_cleaned = df_cleaned[["Timestamp","ECG1", "ECG2"]]

In [105]:
df_cleaned.head()

,Timestamp,ECG1,ECG2
0,2024-01-01 00:00:00.000,0.16,-0.29
1,2024-01-01 00:00:00.008,0.18,-0.30
2,2024-01-01 00:00:00.016,0.17,-0.32
3,2024-01-01 00:00:00.023,0.18,-0.29
4,2024-01-01 00:00:00.031,0.18,-0.31


In [108]:
import plotly.express as px

fig = px.line(df, x=df_cleaned["Timestamp"], y=df_cleaned["ECG1"])
fig.show()

In [110]:
df_cleaned.to_csv('cleaned_ECG.csv',index=False)